<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Unet_Alignmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
from labelme import utils
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow import keras
from keras import models
import os
import os.path
import cv2
import numpy as np
import json
import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
import PIL
import datetime
from keras import backend as K
from pathlib import Path
from keras.models import load_model
from keras.models import Model
import tensorflow as tf
from keras.layers import BatchNormalization, Input, \
    Activation, Conv2D, MaxPooling2D, Conv2DTranspose, Add, concatenate, Concatenate, UpSampling2D, Dropout
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
# Config CPU,GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

input_shape =(512,384,1) #(960, 1280, 1)
train_region = (130, 1170, 384, 384) #(x, y), with, height
OPTIMIZER = tf.keras.optimizers.Adam()
epochs=10000
BATCH_SIZE=2

def LoadDataToTrain(input_shape=(256,256,1), path='', test_ratio=0.2, Use_Region = False):
    images_path = path
    X_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'train')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    Y_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'label')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]

    train_img_paths, test_img_paths, train_label_paths, test_label_paths = train_test_split(X_path, Y_path,test_size=0.2)
    if Use_Region == True:
        X_train = loadImageCrop(train_img_paths, input_shape=input_shape, islabel=0)
        Y_train = loadImageCrop(train_label_paths, input_shape=input_shape, islabel=1)
        X_test = loadImageCrop(test_img_paths, input_shape=input_shape, islabel=0)
        Y_test = loadImageCrop(test_label_paths, input_shape=input_shape, islabel=1)
    else:
        X_train = loadImages(train_img_paths, input_shape=input_shape, islabel=0)
        Y_train = loadImages(train_label_paths, input_shape=input_shape, islabel=1)
        X_test = loadImages(test_img_paths, input_shape=input_shape, islabel=0)
        Y_test = loadImages(test_label_paths, input_shape=input_shape, islabel=1)
    Y_train = np.where(Y_train < 1, 0.0, 1.0)
    Y_test = np.where(Y_test < 1, 0.0, 1.0)

    X_test = X_test.reshape((X_test.shape[0], input_shape[0], input_shape[1], 1))
    Y_test = Y_test.reshape((Y_test.shape[0], input_shape[0], input_shape[1], 1))
    X_train = X_train.reshape((X_train.shape[0], input_shape[0], input_shape[1], 1))
    Y_train = Y_train.reshape((Y_train.shape[0], input_shape[0], input_shape[1], 1))
    return (X_train, Y_train), (X_test, Y_test)

def LoadDataTrain(input_shape=(256,256,1), path='', test_ratio=0.2):
    X_Train = None
    Y_Train = None
    X_Test = None
    Y_Test = None

    #(X_Train, Y_Train), (X_Test, Y_Test) = LoadDataFromImage(input_shape=input_shape,path=path, test_ratio=test_ratio)
    path_train = r"E:\ToanPT\Data_train\train"
    path_test = r"E:\ToanPT\Data_train\test"
    (X_Train, Y_Train) = LoadDataFromImage(input_shape=input_shape, path=path_train, test_ratio=test_ratio)
    (X_Test, Y_Test) = LoadDataFromImage(input_shape=input_shape, path=path_test, test_ratio=test_ratio)

    X_Test = X_Test.reshape((X_Test.shape[0], input_shape[0], input_shape[1], 1))
    Y_Test = Y_Test.reshape((Y_Test.shape[0], input_shape[0], input_shape[1], 1))
    X_Train = X_Train.reshape((X_Train.shape[0], input_shape[0], input_shape[1], 1))
    Y_Train = Y_Train.reshape((Y_Train.shape[0], input_shape[0], input_shape[1], 1))
    return (X_Train, Y_Train), (X_Test, Y_Test)

def LoadDataFromImage(input_shape=(256,256,1), path='', test_ratio=0.2):
    images_path = path

    X_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'train')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    Y_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'label')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    #Xu dung crop vung anh
    #X = loadImageCrop(X_path, train_region=train_region, input_shape=input_shape, islabel=0)
    #Y = loadImageCrop(Y_path, train_region=train_region, input_shape=input_shape, islabel=1)

    X = loadImages(X_path, input_shape=input_shape, islabel=0)
    Y = loadImages(Y_path, input_shape=input_shape, islabel=1)

    Y = np.where(Y < 0.5, 0.0, 1.0)
    """
    (n, _, _) = X.shape
    permutate = np.random.permutation(n)
    X = X[permutate, :, :]
    Y = Y[permutate, :, :]
    n_test = int(n * test_ratio)
    X_test = X[0:n_test, :, :]
    Y_test = Y[0:n_test, :, :]
    X_train = X[n_test:n, :, :]
    Y_train = Y[n_test:n, :, :]
    """
    return (X,Y)  #(X_train, Y_train), (X_test, Y_test)

def loadImages(images_path, input_shape, islabel=0):
    data = []
    print("**************")
    for file in images_path:
        print("Load file:" + file)
        im = cv2.imread(file)
        d = len(im.shape)
        if d == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        # Crop Image
        im = cv2.resize(im, (input_shape[1], input_shape[0]))
        im = np.array(im)
        im = im.astype('float32')
        temp = im / 255.0
        if islabel ==1 :
            print(temp.shape)
            print(np.sum(temp))
        data.append(temp)
    X = np.array(data)
    return X

def loadImageCrop(images_path, train_region, input_shape, islabel=0):
    data = []

    for file in images_path:
        print("Load file:" + file)
        im = cv2.imread(file)
        d = len(im.shape)
        if d == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        # Crop Image
        crop_img = im[train_region[1]:train_region[1] + train_region[3],
                   train_region[0]:train_region[0] + train_region[2]]

        crop_img = cv2.resize(crop_img, (input_shape[1], input_shape[0]))

        crop_img = np.array(crop_img)
        crop_img = crop_img.astype('float32')
        temp = crop_img / 255.0
        if islabel ==1 and np.sum(temp) <500:
            temp[:,:]=0.0

        data.append(temp)
    X = np.array(data)
    return X

def bn_Conv2d(x, filters=16, kernel_size=(3, 3), padding='same', strides=(1, 1), dilation_rate=(1, 1), name='Conv', activation='relu'):
    y = Conv2D(filters=filters, kernel_size=kernel_size,
               kernel_initializer='he_normal',
               bias_initializer='he_normal',
               #kernel_regularizer=kernel_regularizer,
               #activity_regularizer=tf.keras.regularizers.L2(1e-5),
               padding=padding,
               use_bias=True,
               strides=strides, dilation_rate=dilation_rate,)(x)
    y = BatchNormalization()(y)
    y = Activation(activation)(y)
    return y
def bn_Conv2DTranspose(x, filters=16, kernel_size=(3, 3), strides=(2, 2), padding='same', dilation_rate=(1, 1),
                       name='transpose', activation='relu'):
    up1 = Conv2DTranspose(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                          dilation_rate=dilation_rate, name=name)(x)
    up1 = BatchNormalization()(up1)
    up1 = Activation(activation)(up1)
    return up1

def AT_UNET_Seq(input_shape):
    inputs = Input(shape=input_shape)
    input_block= inputs
    p=[]
    for i in range(6):
        en1 = bn_Conv2d(x=input_block, filters=2**i, kernel_size=(3,3), name='Enc_{0}_1'.format(i+1), dilation_rate=(1,1))
        en2 = bn_Conv2d(x=en1, filters=2 ** (i+2), kernel_size=(3,3), name='Enc_{0}_2'.format(i + 1), dilation_rate=(2,2))
        p.append(MaxPooling2D(pool_size=(2,2))(en2))

        input_block=p[i]
    trans = bn_Conv2d(x=en2, filters=128, kernel_size=(3,3), strides=(2,2), dilation_rate=(1,1), name='transfer')
    de = trans
    for i in range(5):
        de= bn_Conv2DTranspose(x=de, filters=2**(6-i), kernel_size=(3,3), name='Dec_{0}'.format(i+1))
        de= Add()([de, p[4-i]])
    outputs = bn_Conv2DTranspose(x=de, filters=1, kernel_size=(3,3), name='Dec_6', activation='sigmoid')
    model = Model(inputs=[inputs], outputs=[outputs], name='AT_UNET')
    return model

def AT_UNET(input_shape):
    inputs = Input(shape=input_shape)
    input_block= inputs
    p=[]
    for i in range(6):
        en = bn_Conv2d(x=input_block, filters=2**i, kernel_size=(3,3), name='Enc_{0}_1'.format(i+1), dilation_rate=(1,1))
        en = bn_Conv2d(x=en, filters=2 ** (i+1), kernel_size=(3,3), name='Enc_{0}_2'.format(i + 1), dilation_rate=(2,2))
        en = bn_Conv2d(x=en, filters=2 ** (i+2), kernel_size=(3,3), name='Enc_{0}_3'.format(i + 1), dilation_rate=(3,3))
        p.append(MaxPooling2D(pool_size=(2,2))(en))
        input_block=p[i]
    trans = bn_Conv2d(x=en, filters=128, kernel_size=(3,3), strides=(2,2), dilation_rate=(1,1), name='transfer')
    de = trans
    for i in range(5):
        de= bn_Conv2DTranspose(x=de, filters=2**(6-i), kernel_size=(3,3), name='Dec_{0}'.format(i+1))
        de= Add()([de, p[4-i]])
    outputs = bn_Conv2DTranspose(x=de, filters=1, kernel_size=(3,3), name='Dec_6', activation='sigmoid')
    model = Model(inputs=[inputs], outputs=[outputs], name='AT_UNET')
    return model

def New_AT_UNET(input_shape):
    inputs = Input(shape=input_shape)
    input_block= inputs
    p=[]
    for i in range(6):
        en1 = bn_Conv2d(x=input_block, filters=2**i,
                        kernel_size=(3,3), name='Enc_{0}_1'.format(i+1), dilation_rate=(1,1))
        en2 = bn_Conv2d(x=input_block, filters=2 ** (i+1),
                        kernel_size=(3,3), name='Enc_{0}_2'.format(i + 1), dilation_rate=(2,2))
        en3 = bn_Conv2d(x=input_block, filters=2 ** (i+2),
                        kernel_size=(3,3), name='Enc_{0}_3'.format(i + 1), dilation_rate=(3,3))
        en = concatenate(inputs=[en1, en2, en3],axis=-1)
        en= bn_Conv2d(x=en, filters=2 ** (i+2),
                      kernel_size=(1,1), dilation_rate=(1,1), name='Enc_add_{0}'.format(i+1))
        p.append(MaxPooling2D(pool_size=(2,2))(en))

        input_block=p[i]
    trans = bn_Conv2d(x=en, filters=128, kernel_size=(3,3), strides=(2,2), dilation_rate=(1,1), name='transfer')
    de = trans
    for i in range(5):
        de= bn_Conv2DTranspose(x=de, filters=2**(6-i), kernel_size=(3,3), name='Dec_{0}'.format(i+1))
        de= Add()([de, p[4-i]])
    outputs = bn_Conv2DTranspose(x=de, filters=1, kernel_size=(3,3), name='Dec_6', activation='sigmoid')
    model = Model(inputs=[inputs], outputs=[outputs], name='AT_UNET')
    return model

"""Dinh nghia model U-NET GOC"""
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters,3, padding='same')(inputs)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return  x
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPooling2D((2,2))(x)
    return x, p

def decoder_block(inputs,skip_features ,num_filters):
    x= Conv2DTranspose(num_filters,(2,2), strides=2, padding='same')(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x
def buid_unet(input_shape):
    inputs = Input(input_shape)
    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    """Bridge"""
    b1 = conv_block(p4, 1024)
    """Decoder"""
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    """output"""
    outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)
    model = Model(inputs, outputs, name="U-NET")
    return  model


def unet(pretrained_weights=None, input_size=(256, 256, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=OPTIMIZER, loss='binary_crossentropy', metrics=['accuracy'])

    # model.summary()
    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

print("Image loading: ")

path= r'E:\2.AssyV3_DBending\data'
model_save_path = r'E:\ToanPT\1.Code_train_Unet\models'
path_save = r'E:\2.AssyV3_DBending\test\image_detect'

def train():
    (x_train, y_train), (x_test, y_test) = LoadDataToTrain(input_shape=input_shape,path=path, Use_Region=False)

    model =AT_UNET(input_shape=input_shape) #AT_UNET(input_shape=input_shape)
    model.compile(optimizer=OPTIMIZER, loss = 'binary_crossentropy', metrics=['accuracy'])
    model.summary()

    class SaveModel(keras.callbacks.Callback):
        def __init__(self, path=None):
            self.Path = path
        def on_epoch_end(self, epoch, logs={}):
            loss = float(logs["loss"])
            acc = float(logs["accuracy"])
            val_loss = float(logs["val_loss"])
            val_acc = float(logs["val_accuracy"])
            filename = "Small_weights-epc {0} -loss {1:.4f} -acc {2:.3f} -val_loss {3:.4f} -val_acc {4:.3f}.h5".format(
                epoch, loss, acc, val_loss, val_acc)

            savepath = os.path.join(self.Path, filename)
            #if acc > 0.995 and val_acc > 0.995:
            self.model.save_weights(filepath=savepath)

    callbackmonitor = [
            keras.callbacks.TensorBoard(log_dir='./logs'),
            SaveModel("./models/")
       ]

    callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs'),
                      ModelCheckpoint(
                          os.path.join(model_save_path,
                                       'unet-AssyDBending-{epoch:03d}--{loss:.6f}-{accuracy:.6f}--{val_loss:.6f}-{val_accuracy:.6f}.h5'),
                          monitor='val_accuracy', save_best_only=False,
                          save_weights_only=False, period=10, mode='auto', verbose=0),
                     ]

    model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=1,
                  epochs=epochs, batch_size=BATCH_SIZE, shuffle=True,
                  callbacks=callbacks)

def buildImage(x, y, input_shape):
    try:
        x = np.reshape(x, input_shape)
        y = np.reshape(y, input_shape)
        x = x * 255
        x = cv2.cvtColor(x, cv2.COLOR_GRAY2RGB)
        y = y * 255
        x = x.astype('uint8')
        y = y.astype('uint8')
        cv2.threshold(y, 200, 255, cv2.THRESH_BINARY, y)
        kernel = np.ones((3, 3), np.uint8)
        raw = y
        # y = cv2.erode(y, kernel=kernel, iterations=3)

        y = cv2.erode(y, kernel=kernel, iterations=1)
        # y = cv2.dilate(y, kernel=kernel, iterations=1)
        contours, _ = cv2.findContours(y, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        index = -1
        i = -1
        area = 0
        contours_filtered = []
        for cnt in contours:
            i = i + 1
            contour_area = cv2.contourArea(cnt)
            if contour_area > area:
                index = i
                area = contour_area
        x_top, y_top, w, h = cv2.boundingRect(contours[index])
        x = cv2.rectangle(x, (x_top, y_top), (x_top + w, y_top + h), (0, 0, 255), 1)
        return x, y, raw
    except:
        print(" Buil Image Exception")

def Predict():

    # image_path=r'E:\1.AssyV3_Align\DB1'
    image_path=r'E:\2.AssyV3_DBending\test'
    # model_file=r'E:\ToanPT\1.Code_train_Unet\models\unet-10000--0.000000-1.000000--0.013596-0.999356.h5'
    model_file=r'E:\ToanPT\1.Code_train_Unet\models\unet-AssyDBending-540--0.009974-0.999938--0.017954-0.999532.h5'
    model = load_model(model_file)  #models.load_model(model_file)

    print("Load model complete")

    Img_path = [os.path.join(root, file) for root, _, files in
                os.walk(os.path.join(image_path, 'train')) for file in files if
                (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    for file_name in Img_path:
        print("Load file:" + file_name)
        im = cv2.imread(file_name)
        d = len(im.shape)
        if d == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)


        # Crop Image
        """
        crop_img = im[train_region[1]:train_region[1] + train_region[3],
                   train_region[0]:train_region[0] + train_region[2]]
        crop_img = cv2.resize(crop_img, (input_shape[1], input_shape[0]))
        crop_img = np.array(crop_img)
        crop_img = crop_img.astype('float32')
        crop_img = crop_img / 255.0
        crop_img = crop_img.reshape((1, input_shape[0], input_shape[1]))
        """
        X = cv2.resize(im,(input_shape[1], input_shape[0]))
        X = np.array(X)
        X= X.astype('float32')
        X = X / 255.0
        X = X.reshape((1, input_shape[0], input_shape[1]))

        y = model.predict(X)

        img, mask, raw_predict = buildImage(X, y, input_shape)
        """
        y = np.reshape(y, input_shape)
        y = y * 255
        y = cv2.cvtColor(y, cv2.COLOR_GRAY2RGB)
        y = y.astype('uint8')
        """

        # plt.imshow(mask)
        # plt.show()

        directory = os.path.dirname(file_name)
        path_save = os.path.join(directory, "Image_Predict")
        print(path_save)
        if not os.path.exists(path_save):
            os.makedirs(path_save)
        cv2.imwrite(os.path.join(path_save, Path(file_name).stem + "_img_predict_draw.jpg"), img)
        cv2.imwrite(os.path.join(path_save, Path(file_name).stem + "_img_predict_mask.jpg"), mask)
        cv2.imwrite(os.path.join(path_save, Path(file_name).stem + "_img_predict_raw.jpg"), raw_predict)

    print('Predict Ok')

def PredictRegion():

    image_path = r'E:\ToanPT\Data_train\test'
    model_file = r'E:\ToanPT\1.Code_train_Unet\2.2.0_071022AtcModel_COP_Resize_Left_DataArg=False_TrainRegion[200x60,1024x1536]-3000--0.000000-1.000000--0.000967-0.999933.h5'
    model = load_model(model_file)  # models.load_model(model_file)
    print("Load model complete")

    Img_path = [os.path.join(root, file) for root, _, files in
                os.walk(os.path.join(image_path, 'train')) for file in files if
                (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    for file_name in Img_path:
        print("Load file:" + file_name)
        im = cv2.imread(file_name)
        d = len(im.shape)
        if d == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        # Crop Image
        crop_img = im[train_region[1]:train_region[1] + train_region[3],
                   train_region[0]:train_region[0] + train_region[2]]
        crop_img = cv2.resize(crop_img, (input_shape[1], input_shape[0]))
        crop_img = np.array(crop_img)
        crop_img = crop_img.astype('float32')
        crop_img = crop_img / 255.0
        crop_img = crop_img.reshape((1, input_shape[0], input_shape[1]))

        X = np.array(crop_img)
        y = model.predict(X)

        img, mask, raw_predict = buildImage(X, y, input_shape)
        directory = os.path.dirname(file_name)
        path_save = os.path.join(directory, "Image_Predict")
        print(path_save)
        if not os.path.exists(path_save):
            os.makedirs(path_save)
        cv2.imwrite(os.path.join(path_save, Path(file_name).stem + "_img_predict_draw.jpg"), img)
        cv2.imwrite(os.path.join(path_save, Path(file_name).stem + "_img_predict_mask.jpg"), mask)
        cv2.imwrite(os.path.join(path_save, Path(file_name).stem + "_img_predict_raw.jpg"), raw_predict)
    print('Predict Ok')


# Predict()
#train()

model=AT_UNET_Seq((256,256,1))
model.summary()
